# Enriching NYC Airbnb Dataset

## WEB SCRAPING

In [81]:
import numpy as np
import pandas as pd
import src.clean_utils as cu
import requests
from bs4 import BeautifulSoup
import re

ModuleNotFoundError: No module named 'src'

## API

In [53]:
import requests

In [60]:
url = "https://data.cityofnewyork.us/resource/pitm-atqc.csv"

In [66]:
url_params = "https://data.cityofnewyork.us/resource/pitm-atqc.json"

In [67]:
response_params = requests.get(url_params).json()

In [76]:
data_api = pd.DataFrame(response_params)
data_api.head()

,objectid,globalid,seating_interest_sidewalk,restaurant_name,legal_business_name,doing_business_as_dba,bulding_number,street,borough,zip,...,community_board,council_district,census_tract,bin,bbl,nta,roadway_dimensions_length,roadway_dimensions_width,roadway_dimensions_area,landmarkdistrict_terms
0,100,c4b3155b-31a0-4e95-846f-fce09f245437,sidewalk,Pomp and Circumstance Hospitality,Pomp and Circumstance Hospitality LLC,Pomp and Circumstance Hospitality LLC,577,Lorimer Street,Brooklyn,11211,...,1,34,501,3068653,3027560028,East Williamsburg,NaN,NaN,NaN,NaN
1,1000,753495d8-4429-43e5-85a3-dcf6230ef749,both,Charm Kao,193 Schemerhorn INC,Charm Kao,193,Schermerhorn St.,Brooklyn,11201,...,2,33,37,3000493,3001640041,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,24,8,192,NaN
2,10000,{3842B5C5-EF04-41A4-8216-D6EA627DCE5E},openstreets,SAKE BAR HAGI 46,"HAMA NEW YORK, INC.",SAKE BAR HAGI 46,358,W. 46TH STREET,Manhattan,10036,...,4,3,121,1025025,1010360057,Clinton,NaN,NaN,NaN,NaN
3,10001,{C212A0FC-C115-4425-8F95-931B12C5F86A},openstreets,Yum yum too,Boythaicorp,Boythaicorp,662,9ave,Manhattan,10036,...,4,3,127,1025038,1010370001,Clinton,NaN,NaN,NaN,NaN
4,10002,{DA48265D-7730-416F-8E1C-EBC8C8ACE2C2},openstreets,Xochil Pizza Corp,Xochil Pizza Corp,Xochil Pizza Corp,4632,5th Avenue,Brooklyn,11220,...,7,38,80,NaN,NaN,Sunset Park West,NaN,NaN,NaN,NaN


In [71]:
data_api.dtypes

objectid                         object
globalid                         object
seating_interest_sidewalk        object
restaurant_name                  object
legal_business_name              object
doing_business_as_dba            object
bulding_number                   object
street                           object
borough                          object
zip                              object
business_address                 object
food_service_establishment       object
sidewalk_dimensions_length       object
sidewalk_dimensions_width        object
sidewalk_dimensions_area         object
approved_for_sidewalk_seating    object
approved_for_roadway_seating     object
qualify_alcohol                  object
sla_serial_number                object
sla_license_type                 object
landmark_district_or_building    object
healthcompliance_terms           object
time_of_submission               object
latitude                         object
longitude                        object


In [73]:
data_api.isnull().sum().apply(lambda x: x/data_api.shape[0]).sort_values(ascending=False)

landmarkdistrict_terms           0.848
sla_serial_number                0.405
sla_license_type                 0.405
roadway_dimensions_width         0.378
roadway_dimensions_length        0.378
roadway_dimensions_area          0.378
sidewalk_dimensions_length       0.217
sidewalk_dimensions_width        0.217
sidewalk_dimensions_area         0.217
bin                              0.091
bbl                              0.091
longitude                        0.085
latitude                         0.085
council_district                 0.085
census_tract                     0.085
nta                              0.085
community_board                  0.085
doing_business_as_dba            0.000
bulding_number                   0.000
globalid                         0.000
legal_business_name              0.000
restaurant_name                  0.000
seating_interest_sidewalk        0.000
borough                          0.000
street                           0.000
qualify_alcohol          

In [74]:
data_api.isnull().sum(axis=1).apply(lambda x: x/data_api.shape[1]).sort_values(ascending=False)

17     0.485714
61     0.428571
279    0.428571
81     0.400000
652    0.400000
         ...   
197    0.000000
752    0.000000
133    0.000000
296    0.000000
522    0.000000
Length: 1000, dtype: float64

In [75]:
data_api.duplicated().sum()

0

In [79]:
col_list = ["restaurant_name", "borough", "street","longitude","latitude"]
data_api_clean = data_api[col_list]
data_api_clean

,restaurant_name,borough,street,longitude,latitude
0,Pomp and Circumstance Hospitality,Brooklyn,Lorimer Street,-73.949416,40.714264
1,Charm Kao,Brooklyn,Schermerhorn St.,-73.986352,40.689107
2,SAKE BAR HAGI 46,Manhattan,W. 46TH STREET,-73.989528,40.760463
3,Yum yum too,Manhattan,9ave,-73.990683,40.761081
4,Xochil Pizza Corp,Brooklyn,5th Avenue,-74.008216,40.64714
...,...,...,...,...,...
995,DONT TELL MAMA,Manhattan,343 WEST 46 STREET,-73.989196,40.76034
996,OLD MAN HUSTLE,Manhattan,39 ESSEX STREET,-73.989467,40.716135
997,Tsurutontan Udon Noodle Brasserie,Manhattan,48th,NaN,NaN
998,PHO PLUS,Queens,13351 37TH AVE,-73.83328,40.760757
